## Building a Data Model - Star Schema

Note, this would be better managed in a "gold" directory as a .py file, but I am housing it here so it is more easily readable and consolidated with the other notebooks.

In [1]:
# Reading in combined data for us to separate out
import os 

# Move the execution of the folder up one directory
os.chdir('..')

from pyspark.sql import SparkSession
from etl.read_normalize import ingest_parquet
from pyspark.sql.functions import monotonically_increasing_id, trim

spark = SparkSession.builder.appName("OlympicCountryDataPipeline").getOrCreate()

df_denormalized = ingest_parquet(
    input_path = "datasets/countries_olympics_join.parquet"
    , spark = spark
)

/Users/coleparker/Documents/holman-bi-dev-test/venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
25/01/16 07:18:07 WARN Utils: Your hostname, Coles-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.235 instead (on interface en0)
25/01/16 07:18:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/16 07:18:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/16 07:18:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/16 07:18:10 WARN Utils: Service 'SparkUI' could not bind on po

In [2]:
df_denormalized.show(10)

25/01/16 07:18:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+----+------+------+-----+----+------------+--------------------+-----------+----------+---------------------+--------------------------+-------------+--------------------------------+--------------+----------------+---------------+--------------+-------------+-------------+-------+---------+---------+-----------+--------+-------+
|Country_Code|Gold|Silver|Bronze|Total|Year|Country_Name|              Region| Population|Area_sq_mi|Pop_Density_per_sq_mi|Coastline_coast_area_ratio|Net_migration|Infant_mortality_per_1000_births|GDP_per_capita|Literacy_percent|Phones_per_1000|Arable_percent|Crops_percent|Other_percent|Climate|Birthrate|Deathrate|Agriculture|Industry|Service|
+------------+----+------+------+-----+----+------------+--------------------+-----------+----------+---------------------+--------------------------+-------------+--------------------------------+--------------+----------------+---------------+--------------+-------------+-------------+-------+---------+--

## Creating Star Schema Tables - Olympics & Countries

We will first focus on creating fact and dimemnsion tables with our olympics and countries dataset. I am starting with the denormalized table to show the most steps in taking a very wide table and transforming into a star schema structure.

In [3]:
# sequence of dimension tables - starting with countries.

dim_country_df = df_denormalized.select(
    "Country_Code", 
    "Country_Name", 
    "Population", 
    "Area_sq_mi", 
    "Pop_Density_per_sq_mi",
    "Coastline_coast_area_ratio",
    "Net_migration",
    "Infant_mortality_per_1000_births",
    "GDP_per_capita",
    "Literacy_percent",
    "Phones_per_1000",
    "Arable_percent",
    "Crops_percent",
    "Other_percent",
    "Climate",
    "Birthrate",
    "Deathrate",
    "Agriculture",
    "Industry",
    "Service").distinct()

# Generating a unique ID for each region to sit as its own dimension table!
dim_region_df = (df_denormalized.select("Region")
                 .distinct()
                 .withColumn("Region_ID", monotonically_increasing_id())
                 )

# Creating our fact_table to use our ID columns. Any columns that will act as 
# foreign keys (such as region) are converted to the proper ID. The other dimension
# attributes are dropped. 
fact_olympics = (df_denormalized
                   .join(dim_region_df, on = "Region", how = "inner")
                   ).drop("Region", "Country_Name", "Population", "Area_sq_mi", \
                    "Pop_Density_per_sq_mi", "Coastline_coast_area_ratio", "Net_migration", \
                    "Infant_mortality_per_1000_births", "GDP_per_capita", "Literacy_percent", \
                    "Phones_per_1000", "Arable_percent", "Crops_percent", "Other_percent", \
                    "Climate", "Birthrate", "Deathrate", "Agriculture", "Industry","Service")



In [4]:
dim_region_df.show()

+--------------------+---------+
|              Region|Region_ID|
+--------------------+---------+
|BALTICS          ...|        0|
|C.W. OF IND. STATES |        1|
|ASIA (EX. NEAR EA...|        2|
|WESTERN EUROPE   ...|        3|
|NEAR EAST        ...|        4|
|EASTERN EUROPE   ...|        5|
|OCEANIA          ...|        6|
|SUB-SAHARAN AFRIC...|        7|
|NORTHERN AFRICA  ...|        8|
|LATIN AMER. & CAR...|        9|
|NORTHERN AMERICA ...|       10|
+--------------------+---------+



Printing out the schema of our fact and dimension tables so we can see how they would join! In this example, the Region_ID could join the fact olympics table to the region dimension. Also, the Country_Code acts as a primary key of the country dimension and joins to the fact olympics table. 

In [22]:
print("Fact - Olympics")
fact_olympics.printSchema()

print("Dim - Region")
dim_region_df.printSchema()

print("Dim - Country")
dim_country_df.printSchema()

Fact - Olympics
root
 |-- Country_Code: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Year: string (nullable = true)
 |-- Region_ID: long (nullable = false)

Dim - Region
root
 |-- Region: string (nullable = true)
 |-- Region_ID: long (nullable = false)

Dim - Country
root
 |-- Country_Code: string (nullable = true)
 |-- Country_Name: string (nullable = true)
 |-- Population: double (nullable = true)
 |-- Area_sq_mi: double (nullable = true)
 |-- Pop_Density_per_sq_mi: double (nullable = true)
 |-- Coastline_coast_area_ratio: double (nullable = true)
 |-- Net_migration: double (nullable = true)
 |-- Infant_mortality_per_1000_births: double (nullable = true)
 |-- GDP_per_capita: double (nullable = true)
 |-- Literacy_percent: double (nullable = true)
 |-- Phones_per_1000: double (nullable = true)
 |-- Arable_percent: double (nullable = true)
 |-- Crops

## Adding More Fact & Dimension Tables

#### Reading in our data

I choose to read in the country age and country crime index datasets. Note that these will also have primary keys of Country_Code and can be joined to our fact olympics table.

In [5]:
df_age = spark.read.csv(
    "datasets/countries_health/Coutries age structure.csv",  
    header=True,              
    inferSchema=True         
)

df_crime = spark.read.csv(
    "datasets/countries_health/Crime index by countries 2020.csv",  
    header=True,              
    inferSchema=True         
)

df_age.show(10)
df_crime.show(10)

+--------+-----------------+------------------+------------------+
| Country|Age 0 to 14 Years|Age 15 to 64 Years|Age above 65 Years|
+--------+-----------------+------------------+------------------+
|   Japan|           12.90%|            60.10%|               27%|
|   Italy|           13.50%|            63.50%|               23%|
|Portugal|           13.60%|            64.90%|               22%|
| Germany|           13.10%|            65.50%|               22%|
| Finland|           16.40%|            62.40%|               21%|
|Bulgaria|           14.20%|            65.00%|               21%|
|  Greece|           14.20%|            65.40%|               20%|
|  Sweden|           17.50%|            62.50%|               20%|
|  Latvia|           15.40%|            64.80%|               20%|
| Denmark|           16.50%|            63.80%|               20%|
+--------+-----------------+------------------+------------------+
only showing top 10 rows

+-------------------+-----------+---

#### Creating our dimension tables to add to star schema

In [6]:
dim_age = (df_age
           .join(dim_country_df, trim(df_age["Country"]) == trim(dim_country_df["Country_Name"]), how = "inner")
           ).select("Country_Code", "Age 0 to 14 Years", "Age 15 to 64 Years", "Age above 65 Years")

dim_crime = (df_crime
           .join(dim_country_df, trim(df_crime["Country"]) == trim(dim_country_df["Country_Name"]), how = "inner")
           ).select("Country_Code", "Crime Index", "Safety Index")

Note: In our final star schema, since all of the data we have is around countries, the country code is the unique ID that is used to join our tables through primary and foreign keys. In our final structure, the region dimension can be joined to the fact table through the Region_ID and all other dimensions can be joined to the fact table through the Country_Code. 

In [41]:
print("Fact - Olympics")
fact_olympics.printSchema()

print("Dim - Region")
dim_region_df.printSchema()

print("Dim - Country")
dim_country_df.printSchema()

print("Dim - Age")
dim_age.printSchema()

print("Dim - Crime")
dim_crime.printSchema()

Fact - Olympics
root
 |-- Country_Code: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Year: string (nullable = true)
 |-- Region_ID: long (nullable = false)

Dim - Region
root
 |-- Region: string (nullable = true)
 |-- Region_ID: long (nullable = false)

Dim - Country
root
 |-- Country_Code: string (nullable = true)
 |-- Country_Name: string (nullable = true)
 |-- Population: double (nullable = true)
 |-- Area_sq_mi: double (nullable = true)
 |-- Pop_Density_per_sq_mi: double (nullable = true)
 |-- Coastline_coast_area_ratio: double (nullable = true)
 |-- Net_migration: double (nullable = true)
 |-- Infant_mortality_per_1000_births: double (nullable = true)
 |-- GDP_per_capita: double (nullable = true)
 |-- Literacy_percent: double (nullable = true)
 |-- Phones_per_1000: double (nullable = true)
 |-- Arable_percent: double (nullable = true)
 |-- Crops